<a href="https://colab.research.google.com/github/andreafortuna/stable-diffusion-colab/blob/main/stable-diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
huggingface_token = "" #@param {type:"string"}
%pip install --quiet --upgrade diffusers transformers scipy mediapy ftfy python-slugify

from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(huggingface_token)

from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085,beta_end=0.012, beta_schedule="scaled_linear",num_train_timesteps=1000)
#scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)
#scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
#scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", use_auth_token=True,hub_token = huggingface_token)
pipe.safety_checker = lambda images, clip_input: (images, False)

pipe = pipe.to("cuda")



In [ ]:
#@title Image generation
prompt = "paint of Spiderman in Picasso style" #@param {type:"string"}
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"integer"}
image_size = 512 #@param ["320", "512", "640"] {type:"raw"}
#prompt = input("Image description: ")
num_images = 1
from slugify import slugify

prompts = [ prompt ] * num_images
with autocast("cuda"):
    images = pipe(prompts, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps,height=image_size, width=image_size).images 
    
media.show_images(images)
image_name = slugify(prompt)
images[0].save(image_name + ".jpg")